# Time Series

## Imports

In [2]:
import pandas as pd
import plotly
from bubbly import bubbleplot
plotly.offline.init_notebook_mode(connected=True)

## Functions

In [12]:
def subsetData(df, start_date, end_date):
    df = df[(df["Date"] >= start_date) & (
    df["Date"] <= end_date)].reset_index().drop(columns=["index"])

    df.replace(2400, 0, inplace=True)

    df.sort_values(by=["Hour", "Sensor"],inplace=True)
    
    return df

In [18]:
def plotTimeSeries(df):
    
    #Variables
    x_column = 'SensorLongitude'
    y_column = 'SensorLatitude'
    bubble_column = 'Sensor'
    time_column = 'Hour'
    size_column = 'CrowdednessCount' 
    date = pd.Timestamp.strftime(df["Date"][0], format="%Y-%m-%d")

    #Initialize grid
    grid = pd.DataFrame()
    
    figure = bubbleplot(dataset=df, x_column=x_column, y_column=y_column, 
        bubble_column=bubble_column, size_column=size_column, time_column=time_column, color_column=bubble_column,
        x_title="Sensor Longitude", y_title="Sensor Latitude", title='Crowdedness Counts Amsterdam - ' + date,
        x_logscale=False, scale_bubble=3, height=650, x_range=[min(df[x_column])-0.001, max(df[x_column])+0.001],
                       y_range=[min(df[y_column])-0.001, max(df[y_column])+0.001])

    plotly.offline.iplot(figure)

## Import File

In [14]:
dataset = pd.read_csv("../../../Data_thesis/Full_Datasets/Full.csv")
dataset = dataset[["Date", "Hour", "Sensor", "SensorLongitude", "SensorLatitude", "CrowdednessCount"]]
dataset["Date"] = pd.to_datetime(dataset["Date"], format="%Y-%m-%d")

## TimeSeries Plot

In [15]:
start_date = pd.Timestamp(2019, 1, 1)
end_date = pd.Timestamp(2019, 1, 1)

In [16]:
dataset = subsetData(dataset, start_date, end_date)
plotTimeSeries(dataset)